In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt

from symr.helpers import r2_over_threshold, r2_auc, plot_r2_over_threshold

import pandas as pd
import sympy as sp
import math

In [ ]:
# modify the results path and string tag to load different csvs
root_path = ".."
results_path = os.path.join(root_path, "results")
string_tag = ["860200_partial.csv", "b1"]

In [ ]:
string_tag

In [ ]:
# interpolation version of table 1


list_dir = os.listdir(results_path)

active_dir = []

numerical_table = {}

for filename in list_dir:
    toss = 0
    for str_tag in string_tag:
        if str_tag not in filename:
            toss += 1
    if not(toss):
        active_dir.append(filename)
        
        print(filename, toss)

for table_tag in ["nguyen", "ab_complex"]:

    first_row = None
    
    latex_table = "\\begin{table}[h]\n"
    latex_table += "\\center{ \n"
    latex_table += "\\begin{tabular}{l | c | c | c |}\n"
    
    if first_row is None:
    
        first_row = "method & nmse$_{i.d.} \pm \sigma$ & $R^2_{i.d.} > 0.95 \pm \sigma$ &"\
                "  np.isclose$_{i.d.}\pm \sigma$ \\\\ \n"
        first_row += "\hline \\\\ \n"
        latex_table += first_row
        
    for filename in active_dir:

        filepath = os.path.join(results_path, filename)

        df = pd.read_csv(filepath)
        methods = df["method"].unique()[1:]

        for method in methods:
            
            if table_tag in filename:

                in_success = " False" == df.loc[df["method"] == method]["failed"].to_numpy()

                # isclose
                # this method can handle inf, nan, so we will first take this metric
                # before filtering out non-finite numbers
                in_isclose_raw = df.loc[df["method"] == method][" in_isclose"]
                in_isclose =  in_isclose_raw[in_success].to_numpy()  == " True"



                # R^2 over threshold
                in_r2_over_95_raw = df.loc[df["method"] == method][" in_r2_over_95"]
                in_r2_over_99_raw = df.loc[df["method"] == method][" in_r2_over_99"]

                # should not occur
                if " None" in in_r2_over_95_raw:
                    print("should not have Nones")
                if " None" in in_r2_over_99_raw:
                    print("should not have Nones")

                if(1):
                    in_success[in_r2_over_99_raw == " None"] = 0.0
                    in_success[in_r2_over_95_raw == " None"] = 0.0
                    
                in_r2_over_95 =  in_r2_over_95_raw[in_success].to_numpy()  == " True"
                in_r2_over_99 =  in_r2_over_99_raw[in_success].to_numpy()  == " True"

                # nmse
                in_nmse_raw = df.loc[df["method"] == method][" in_nmse"]
                in_nmse =  in_nmse_raw[in_success].to_numpy(dtype=float)
                # filter out non-finite values

                in_nmse = in_nmse[np.isfinite(in_nmse)]


                table_row = f"{method} (mean/median) & {np.mean(in_nmse):.2e}/{np.median(in_nmse):.2e} $\pm$ {np.std(in_nmse):.3e} &"
                table_row += f" {np.mean(in_r2_over_95):.2f} $\pm$ {np.std(in_r2_over_95):.3f} &"
                #table_row += f" {np.mean(in_r2_over_99):.3e} $\pm$ {np.std(in_r2_over_99):.3e} &"
                table_row += f" {np.mean(in_isclose):.3f} $\pm$ {np.std(in_isclose):.3f} "

                table_row += " \\\\ \n"

                latex_table += table_row


    
    if "nguyen" in table_tag:
        bm_name = "Nguyen"
    elif "ab_complex" in table_tag:
        bm_name = "A.B."
        
    latex_table += """
    \end{tabular}\n
    }
    \caption{
    """
    latex_table += f"Interpolation numerical metrics on {bm_name} equations"
    latex_table += """
    }
    \end{table}
    """
    
    print(latex_table)

In [ ]:
# extrapolation version of table 1

list_dir = os.listdir(results_path)

active_dir = []

numerical_table = {}

for filename in list_dir:
    toss = 0
    for str_tag in string_tag:
        if str_tag not in filename:
            toss += 1
    if not(toss):
        active_dir.append(filename)
        
        print(filename, toss)

for table_tag in ["nguyen", "ab_complex"]:

    first_row = None
    
    latex_table = "\\begin{table}[h]\n"
    latex_table += "\\center{ \n"
    latex_table += "\\begin{tabular}{l | c | c | c |}\n"
    
    if first_row is None:
    
        first_row = "method & nmse$_{i.d.} \pm \sigma$ & $R^2_{i.d.} > 0.95 \pm \sigma$ &"\
                "  np.isclose$_{i.d.}\pm \sigma$ \\\\ \n"
        first_row += "\hline \\\\ \n"
        latex_table += first_row
        
    for filename in active_dir:

        filepath = os.path.join(results_path, filename)

        df = pd.read_csv(filepath)
        methods = df["method"].unique()[1:]
        for method in methods:
            
            if table_tag in filename:

                
                ex_success = " False" == df.loc[df["method"] == method]["failed"].to_numpy()

                # isclose
                # this method can handle inf, nan, so we will first take this metric
                # before filtering out non-finite numbers
                ex_isclose_raw = df.loc[df["method"] == method][" ex_isclose"]
                ex_isclose =  ex_isclose_raw[ex_success].to_numpy()  == " True"

                # R^2 over threshold
                ex_r2_over_95_raw = df.loc[df["method"] == method][" ex_r2_over_95"]
                ex_r2_over_99_raw = df.loc[df["method"] == method][" ex_r2_over_99"]

                # should not occur
                if " None" in ex_r2_over_99_raw:
                    print("should not have Nones")
                if " None" in ex_r2_over_95_raw:
                    print("should not have Nones")

                if(1):
                    ex_success[ex_r2_over_99_raw == " None"] = 0.0
                    ex_success[ex_r2_over_95_raw == " None"] = 0.0

                ex_r2_over_95 =  ex_r2_over_95_raw[ex_success].to_numpy() == " True"
                ex_r2_over_99 =  ex_r2_over_99_raw[ex_success].to_numpy()  == " True"

                # nmse
                ex_nmse_raw = df.loc[df["method"] == method][" in_nmse"]

                ex_nmse =  ex_nmse_raw[ex_success].to_numpy(dtype=float)
                # filter out non-finite values

                ex_nmse = ex_nmse[np.isfinite(ex_nmse)]

                #print(np.std(in_nmse))
                #print(np.var(in_nmse))

                table_row = f"{method} (mean/median) & {np.mean(ex_nmse):.2e}/{np.median(ex_nmse):.2e} $\pm$ {np.std(ex_nmse):.3e} &"
                table_row += f" {np.mean(ex_r2_over_95):.2f} $\pm$ {np.std(ex_r2_over_95):.3f} &"
                #table_row += f" {np.mean(in_r2_over_99):.3e} $\pm$ {np.std(in_r2_over_99):.3e} &"
                table_row += f" {np.mean(ex_isclose):.3f} $\pm$ {np.std(ex_isclose):.3f} "

                table_row += " \\\\ \n"

                latex_table += table_row


    
    if "nguyen" in table_tag:
        bm_name = "Nguyen"
    elif "ab_complex" in table_tag:
        bm_name = "A.B."
        
    latex_table += """
    \end{tabular}\n
    }
    \caption{
    """
    latex_table += f"Extrapolation numerical metrics on {bm_name} equations"
    latex_table += """
    }
    \end{table}
    """
    
    print(latex_table)